# 1. Topology design
We set up the topology with the following details in mind.

- Networks:
  - Host-only: VMnet1 - 192.168.134.0/24
  - NAT: VMnet8 - 192.168.100.0/24

- Ubuntu-desktop-control-station:
  - Connect to VMnet1 at 192.168.134.130
  - Connect to VMnet8 at 192.168.100.147

- GNS3:
  - Router image: i86bi-linux-l3-adventerprisek9-15.5.2T
  - The startup-config is [here](/docs/MEGA-LAB/configs/mega-lab-router-startup-config.txt).
  - Note: this startup-config has to copy in 2 folders in order to work.
    - In the host machine: C:\Users\kimdo\GNS3\configs\mega-lab-router.txt (kimdo is my computer
    name).
    - In the GNS3 vm: /GNS3/projects/

## 1.1. Initial topology
The initial topology is as in figure below.

![Alt_text](/docs/MEGA-LAB/images/00_initial_topology_v1.png)

### Configure EDGE router:
The steps are as follow:
- Generate RSA key for ssh connection
- config e0/0: 192.168.134.101
- config e0/1: 10.15.1.254
- config EIGRP
  ```bash
  conf t
  router eigrp 100
  network 10.0.0.0
  ```

### Configure R1:
The steps are as follow:
- Generate RSA key for ssh connection
- config e0/0: 10.15.1.1
- config lo0: 10.14.1.1/32
- config EIGRP
  ```bash
  conf t
  router eigrp 100
  network 10.0.0.0
  ```

### Add static route in the Ubuntu control station
```bash
ip route add 10.15.1.0/24 via 192.168.134.101
```
Now, the ubuntu desktop can ping 10.15.1.254, but can NOT ping 10.15.1.1.

Then we **NEED** to set default static route and redistribution of the
route under EIGRP.

### Configure static route and redistribute route under EIGRP in EDGE1
Follow the example [here](https://www.cisco.com/c/en/us/support/docs/ip/enhanced-interior-gateway-routing-protocol-eigrp/200279-Configure-Default-route-in-EIGRP.html).

The situation is:
- R1, EDGE1 are configured with EIGRP.
- No EIGRP runs in between EDGE1 and the Ubuntu control station.
- In EDGE1:
```bash
ip route 0.0.0.0 0.0.0.0 Null0
router eigrp 100
redistribute static metric 1000 10 255 1 1500
```

### Configure DHCP server on EDGE1

Follow the example [here](https://www.cisco.com/en/US/docs/ios/12_1t/12_1t5/feature/guide/dt_dhcpa.html#wp1015359).


